In [1]:
#####Robust Network Co-Clustering#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
import itertools
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

#np.random.seed(20)

C:\Users\sana\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [3]:
####データの発生####
##データの設定
k1 = 15   #ユーザーのクラスタ数
k21 = 12   #アイテムの関係オブジェクトのクラスタ数
k22 = 8   #アイテムの無関係オブジェクトのクラスタ数
hh = 10000   #ユーザー数
item = 5000   #アイテム数
pt = np.random.poisson(np.random.gamma(27.5, 1/0.2, hh))   #ユーザーあたりのレコード数
f = np.sum(pt)   #総レコード数

In [4]:
##IDとインデックスの設定
#IDの設定
user_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスの設定
index = np.array(range(f))
user_index = [i for i in range(hh)]
user_vec = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]
    user_vec[i] = np.repeat(1, len(user_index[i]))

In [5]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id = np.zeros(f, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id[user_index[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))
    
#インデックスの設定
item_index = [j for j in range(item)]
item_vec = [j for j in range(item)]
for j in range(item):
    item_index[j] = index[item_id==j]
    item_vec[j] = np.repeat(1, len(item_index[j]))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [6]:
##セグメント割当を生成
#関係有無の潜在変数rを生成
tau = 0.4; taut = tau
r = np.random.binomial(1, tau, item); rt = r
index_r0 = np.array(range(item))[r==0]; n0 = index_r0.shape[0]
index_r1 = np.array(range(item))[r==1]; n1 = index_r1.shape[0]

#ユーザーセグメントを生成
alpha1 = np.random.dirichlet(np.repeat(5.0, k1), 1).reshape(-1); alphat1 = alpha1
Z1 = np.random.multinomial(1, alpha1, hh)
z_vec1 = np.dot(Z1, np.arange(k1))

#アイテムセグメントを生成
alpha21 = np.random.dirichlet(np.repeat(5.0, k21), 1).reshape(-1); alphat21 = alpha21
alpha22 = np.random.dirichlet(np.repeat(5.0, k22), 1).reshape(-1); alphat22 = alpha22
Z21 = np.zeros((hh, k21)); Z22 = np.zeros((hh, k22))
Z21[index_r1, ] = np.random.multinomial(1, alpha21, n1)
Z22[index_r0, ] = np.random.multinomial(1, alpha22, n0)
z_vec21 = np.array(np.dot(Z21, np.arange(k21)), dtype="int")
z_vec22 = np.array(np.dot(Z22, np.arange(k22)), dtype="int")

##ベータ分布からセグメントごとの関係確率を生成
#関係オブジェクトのパラメータを生成
alpha1 = 0.2; beta1 = 0.5
theta = np.random.beta(alpha1, beta1, k1*k21).reshape(k1, k21)
thetat = theta

#無関係オブジェクトのパラメータを生成
alpha2 = 0.6; beta2 = 1.0
phi = np.random.beta(alpha2, beta2, k22)
phit = phi

##関係データを生成
#セグメント割当から関係確率を設定
r_vec = r[item_id]
Prob = r_vec * np.sum(theta[z_vec1[user_id], ] * Z21[item_id, ], axis=1) + (1-r_vec) * phi[z_vec22[item_id]]

#ベルヌーイ分布から関係データを生成
y = np.random.binomial(1, Prob, f)
y_vec = y.reshape(f, 1)
index_y0 = np.array(range(f))[y==0]
index_y1 = np.array(range(f))[y==1]
n0 = np.sum(1-y)
n1 = np.sum(y)

In [7]:
####ギブスサンプリングでRobust Network Co-Clusteringをサンプリング#####
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2  
iter = 0
burnin = 500/keep
disp = 10

In [8]:
##事前分布の設定
#混合率の事前分布
s0 = 0.1; v0 = 0.2
alpha01 = 0.25
alpha02 = 0.25

#モデルパラメータの事前分布
er = 0.00001
beta11 = 0.2; beta12 = 0.2
beta21 = 0.2; beta22 = 0.2

In [9]:
##パラメータの真値
#混合率の真値
tau = taut
alpha1 = alphat1
alpha21 = alphat21
alpha22 = alphat22

#モデルパラメータの真値
theta = thetat
phi = phit

#潜在変数の真値
r = rt 
Zi1 = Z1
Zi21 = Z21
Zi22 = Z22
z_vec1 = np.dot(Zi1, np.arange(k1))
z_vec21 = np.array(np.dot(Zi21, np.arange(k21)), dtype="int")
z_vec22 = np.array(np.dot(Zi22, np.arange(k22)), dtype="int")

In [10]:
##パラメータの初期値
#混合率の初期値
tau = 0.4
alpha1 = np.random.dirichlet(np.repeat(10.0, k1), 1).reshape(-1)
alpha21 = np.random.dirichlet(np.repeat(10.0, k21), 1).reshape(-1)
alpha22 = np.random.dirichlet(np.repeat(10.0, k22), 1).reshape(-1)

#モデルパラメータの初期値
theta = np.random.beta(10.0, 20.0, k1*k21).reshape(k1, k21)
phi = np.random.beta(10.0, 20.0, k22)

#潜在変数の初期値
r = np.random.binomial(1, tau, item)
Zi1 = np.random.multinomial(1, alpha1, hh)
Zi21 = np.random.multinomial(1, alpha21, item) * r.reshape(item, 1)
Zi22 = np.random.multinomial(1, alpha22, item) * (1-r.reshape(item, 1))
z_vec1 = np.dot(Zi1, np.arange(k1))
z_vec21 = np.array(np.dot(Zi21, np.arange(k21)), dtype="int")
z_vec22 = np.array(np.dot(Zi22, np.arange(k22)), dtype="int")

In [11]:
##パラメータの格納用配列
THETA = np.zeros((k1, k21, int(R/keep)))
PHI = np.zeros((int(R/keep), k22))
TAU = np.zeros(int(R/keep))
ALPHA1 = np.zeros((int(R/keep), k1))
ALPHA21 = np.zeros((int(R/keep), k21))
ALPHA22 = np.zeros((int(R/keep), k22))
R_SEG = np.zeros((f))
SEG1 = np.zeros((hh, k1, int(R/keep)))
SEG21 = np.zeros((item, k21, int(R/keep)))
SEG22 = np.zeros((item, k22, int(R/keep)))

In [12]:
##対数尤度の基準値
#ユニグラムモデルの対数尤度
LLst = np.sum(y*np.log(np.mean(y)) + (1-y)*np.log(np.mean(1-y)))
print(LLst)

#真値での対数尤度
r_vec = rt[item_id]
LLbest11 = np.sum(y[r_vec==1] * np.sum(np.log(thetat)[np.dot(Z1, range(k1))[user_id], ] * Z21[item_id, ], axis=1)[r_vec==1])
LLbest12 = np.sum((1-y[r_vec==1]) * np.sum(np.log(1-thetat)[np.dot(Z1, range(k1))[user_id], ] * Z21[item_id, ], axis=1)[r_vec==1])
LLbest21 = np.sum(y[r_vec==0] * np.log(phit)[np.array(np.dot(Z22, range(k22)), dtype="int")[item_id[r_vec==0]]])
LLbest22 = np.sum((1-y[r_vec==0]) * np.log(1-phit)[np.array(np.dot(Z22, range(k22)), dtype="int")[item_id[r_vec==0]]])
LLbest = LLbest11 + LLbest12 + LLbest21 + LLbest22
print(LLbest)

-942183.5486895051
-468400.98409621307


In [13]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##潜在変数rをサンプリング
    #無関係オブジェクトのクラスタごとの対数尤度
    phi_dt = np.zeros((f, k22))
    phi_dt[index_y1, ] = np.log(np.repeat(phi, n1).reshape(n1, k22, order="F"))
    phi_dt[index_y0, ] = np.log(np.repeat(1-phi, n0).reshape(n0, k22, order="F"))

    #関係オブジェクトのクラスタごとの対数尤度
    theta_dt = np.zeros((f, k21))
    theta_dt[index_y1, ] = np.log(theta)[z_vec1[user_id[index_y1]], ]
    theta_dt[index_y0, ] = np.log(1-theta)[z_vec1[user_id[index_y0]], ]

    #クラスタごとの対数尤度の和
    LLi0 = np.zeros((item, k22)); LLi1 = np.zeros((item, k21))
    for j in range(item):
        LLi0[j] = np.dot(item_vec[j], phi_dt[item_index[j], ])
        LLi1[j] = np.dot(item_vec[j], theta_dt[item_index[j], ])

    #潜在変数rの割当確率
    LLi_max = np.max(np.hstack((LLi0, LLi1)), axis=1).reshape(item, 1)
    Li0 = np.sum(np.repeat(alpha22, item).reshape(item, k22) * np.exp(LLi0 - LLi_max), axis=1)
    Li1 = np.sum(np.repeat(alpha21, item).reshape(item, k21) * np.exp(LLi1 - LLi_max), axis=1)
    Prob = np.hstack(((tau*Li1).reshape(item, 1), ((1-tau)*Li0).reshape(item, 1))) / (tau*Li1 + (1-tau)*Li0).reshape(item, 1)
    Prob = ((Prob + er) / np.sum(Prob + er, axis=1).reshape(item, 1))[:, 0]

    #ベルヌーイ分布からrをサンプリング
    r = np.random.binomial(1, Prob, item)
    index_r1 = np.arange(item)[r==1]; rn1 = index_r1.shape[0]
    index_r0 = np.delete(np.arange(item), index_r1); rn0 = index_r0.shape[0]
    
    #ベータ分布から混合率を更新
    tau = np.random.beta(np.sum(r) + s0, item - np.sum(r) + v0, 1)
    

    ##アイテムのクラスタをサンプリング
    #無関係オブジェクトのクラスタをサンプリング
    Zi22 = np.zeros((item, k22))
    LLi_max = np.max(LLi0[index_r0, ], axis=1).reshape(rn0, 1)
    Li = np.repeat(alpha22, rn0).reshape(rn0, k22, order="F") * np.exp(LLi0[index_r0, ] - LLi_max)
    Prob = Li / np.sum(Li, axis=1).reshape(rn0, 1); Prob = (Prob + er) / np.sum(Prob + er, axis=1).reshape(rn0, 1)
    Zi22[index_r0, ] = np.array(rmnom(Prob, rn0, k22, np.arange(rn0)).todense())
    z_vec22 = np.array(np.dot(Zi22, np.arange(k22)), dtype="int")

    #関係オブジェクトのクラスタをサンプリング
    Zi21 = np.zeros((item, k21))
    LLi_max = np.max(LLi1[index_r1, ], axis=1).reshape(rn1, 1)
    Li = np.repeat(alpha21, rn1).reshape(rn1, k21, order="F") * np.exp(LLi1[index_r1, ] - LLi_max)
    Prob = Li / np.sum(Li, axis=1).reshape(rn1, 1); Prob = (Prob + er) / np.sum(Prob + er, axis=1).reshape(rn1, 1)
    Zi21[index_r1, ] = np.array(rmnom(Prob, rn1, k21, np.arange(rn1)).todense())
    z_vec21 = np.array(np.dot(Zi21, np.arange(k21)), dtype="int")

    ##ユーザーのクラスタをサンプリング
    #アイテムのクラスタの条件付き尤度を設定
    theta_T = theta.T
    LLi = np.zeros((hh, k1)); Lho = np.zeros((f, k1))
    Lho[index_y1, ] = np.log(theta_T[z_vec21[item_id[index_y1]], ]) *  r[item_id[index_y1]].reshape(n1, 1) 
    Lho[index_y0, ] = np.log(1 - theta_T[z_vec21[item_id[index_y0]], ]) * r[item_id[index_y0]].reshape(n0, 1)
    for i in range(hh):
        LLi[i, ] = np.dot(user_vec[i], Lho[user_index[i], ])

    #ベルヌーイ分布からクラスタをサンプリング
    Li = np.repeat(alpha1, hh).reshape(hh, k1, order="F") * np.exp(LLi - np.max(LLi, axis=1).reshape(hh, 1))
    Prob = Li / np.sum(Li, axis=1).reshape(hh, 1); Prob = (Prob + er) / np.sum(Prob + er, axis=1).reshape(hh, 1)
    Zi1 = np.array(rmnom(Prob, hh, k1, np.arange(hh)).todense())
    z_vec1 = np.array(np.dot(Zi1, np.arange(k1)), dtype="int")


    ##ベータ分布から関係オブジェクトのパラメータをサンプリング
    #データの設定
    r_vec = r[item_id]
    y_relation = y * r_vec
    Zi_dt1 = Zi1[user_id, ]
    Zi_dt2 = Zi21[item_id, ]

    for j in range(k21):
        #セグメント割当の頻度を数える
        s = np.dot(y_relation * Zi_dt1[:, j], Zi_dt2)
        n = np.dot(r_vec * Zi_dt1[:, j], Zi_dt2)

        #ベータ分布からパラメータをサンプリング
        tau1 = s + beta11
        tau2 = n - s + beta12
        theta[j, ] = np.random.beta(tau1, tau2, k21)

    ##ベータ分布から無関係オブジェクトのパラメータをサンプリング
    #データの設定
    Zi22_dt = Zi22[item_id, ]

    #セグメント割当の頻度を数える
    s = np.sum(y_vec * Zi22_dt, axis=0)
    n = np.sum(Zi22_dt, axis=0)

    #ベータ分布からパラメータをサンプリング
    tau1 = s + beta21
    tau2 = n - s + beta22
    phi = np.random.beta(tau1, tau2, k22)


    ##混合率をサンプリング
    #関係オブジェクトの混合率
    alpha1 = np.random.dirichlet(np.sum(Zi1, axis=0) + alpha01, 1).reshape(-1)
    alpha21 =  np.random.dirichlet(np.sum(Zi21, axis=0) + alpha01, 1).reshape(-1)

    #無関係オブジェクトの混合率
    alpha22 =  np.random.dirichlet(np.sum(Zi22, axis=0) + alpha02, 1).reshape(-1)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        THETA[:, :, mkeep] = theta
        PHI[mkeep, ] = phi
        TAU[mkeep] = tau
        ALPHA1[mkeep, ] = alpha1
        ALPHA21[mkeep, ] = alpha21
        ALPHA22[mkeep, ] = alpha22
        SEG1[:, :, mkeep] = Zi1
        SEG21[:, :, mkeep] = Zi21
        SEG22[:, :, mkeep] = Zi22        
        
    #トピック割当はバーンイン期間を超えたら格納
    if rp%keep==0 & rp >= burnin:
        R_SEG = R_SEG + r
        
    if rp%disp==0:
        #対数尤度の和を計算
        r_vec = r[item_id]
        LL11 = np.sum(y[r_vec==1] * np.sum(np.log(theta)[np.dot(Zi1, range(k1))[user_id], ] * Zi21[item_id, ], axis=1)[r_vec==1])
        LL12 = np.sum((1-y[r_vec==1]) * np.sum(np.log(1-theta)[np.dot(Zi1, range(k1))[user_id], ] * Zi21[item_id, ], axis=1)[r_vec==1])
        LL21 = np.sum(y[r_vec==0] * np.log(phi)[np.array(np.dot(Zi22, range(k22)), dtype="int")[item_id[r_vec==0]]])
        LL22 = np.sum((1-y[r_vec==0]) * np.log(1-phi)[np.array(np.dot(Zi22, range(k22)), dtype="int")[item_id[r_vec==0]]])
        LL = LL11 + LL12 + LL21 + LL22

        #サンプリング結果の表示
        print(rp)
        print(np.round(np.vstack((phi, phit)), 3))
        print(np.round([LL, LLst, LLbest], 1))

0
[[0.723 0.461 0.401 0.074 0.359 0.315 0.404 0.249]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-725895.6 -942183.5 -468401. ]
10
[[0.997 0.79  0.502 0.017 0.304 0.278 0.603 0.232]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-491634.1 -942183.5 -468401. ]
20
[[0.997 0.79  0.517 0.017 0.283 0.279 0.617 0.246]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-489867.8 -942183.5 -468401. ]
30
[[0.997 0.791 0.521 0.009 0.044 0.284 0.618 0.229]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-488709.6 -942183.5 -468401. ]
40
[[0.997 0.79  0.522 0.008 0.039 0.278 0.617 0.224]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-489317.6 -942183.5 -468401. ]
50
[[0.997 0.791 0.525 0.007 0.039 0.278 0.617 0.237]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-488597.4 -942183.5 -468401. ]
60
[[0.997 0.791 0.519 0.007 0.041 0.276 0.618 0.233]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-488384.6 -942183.5 -468401. ]
70
[[0.997 0.79  0.524 0.008 0.04  

C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]
C:\Users\sana\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in log


280
[[0.997 0.793 0.527 0.007 0.041 0.282 0.618 0.239]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480694.6 -942183.5 -468401. ]
290
[[0.997 0.792 0.523 0.006 0.041 0.28  0.617 0.235]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480678.6 -942183.5 -468401. ]
300
[[0.997 0.791 0.523 0.007 0.04  0.279 0.618 0.229]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480634.  -942183.5 -468401. ]
310
[[0.997 0.789 0.522 0.007 0.04  0.273 0.618 0.001]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480608.8 -942183.5 -468401. ]
320
[[0.997 0.792 0.528 0.007 0.039 0.274 0.618 0.738]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480552.8 -942183.5 -468401. ]
330
[[0.997 0.814 0.519 0.007 0.039 0.276 0.616 0.756]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480426.1 -942183.5 -468401. ]
340
[[0.997 0.828 0.521 0.007 0.042 0.274 0.616 0.766]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480188.2 -942183.5 -468401. ]
350
[[0.997 0.823 0.519 0.0

870
[[0.997 0.825 0.518 0.007 0.04  0.274 0.615 0.765]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480323.3 -942183.5 -468401. ]
880
[[0.997 0.829 0.518 0.007 0.041 0.274 0.616 0.762]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480195.6 -942183.5 -468401. ]
890
[[0.997 0.825 0.521 0.007 0.04  0.273 0.617 0.763]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480285.8 -942183.5 -468401. ]
900
[[0.997 0.828 0.519 0.007 0.04  0.275 0.615 0.764]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480317.7 -942183.5 -468401. ]
910
[[0.997 0.826 0.525 0.007 0.04  0.277 0.617 0.763]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480464.7 -942183.5 -468401. ]
920
[[0.997 0.829 0.525 0.007 0.04  0.274 0.616 0.768]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480599.1 -942183.5 -468401. ]
930
[[0.997 0.824 0.523 0.007 0.041 0.273 0.615 0.762]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480381.3 -942183.5 -468401. ]
940
[[0.997 0.826 0.521 0.0

1460
[[0.997 0.828 0.521 0.007 0.04  0.275 0.617 0.763]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480325.1 -942183.5 -468401. ]
1470
[[0.997 0.829 0.522 0.007 0.04  0.276 0.616 0.765]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480471.4 -942183.5 -468401. ]
1480
[[0.997 0.831 0.521 0.007 0.04  0.273 0.618 0.766]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480244.1 -942183.5 -468401. ]
1490
[[0.997 0.827 0.523 0.007 0.041 0.275 0.614 0.763]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480386.5 -942183.5 -468401. ]
1500
[[0.997 0.829 0.524 0.007 0.041 0.273 0.615 0.766]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480286.1 -942183.5 -468401. ]
1510
[[0.997 0.831 0.526 0.007 0.039 0.275 0.617 0.763]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480253.8 -942183.5 -468401. ]
1520
[[0.997 0.822 0.517 0.007 0.039 0.275 0.616 0.764]
 [0.007 0.766 0.528 0.04  0.616 0.997 0.273 0.827]]
[-480190.6 -942183.5 -468401. ]
1530
[[0.997 0.83  0